In [1]:
import numpy as np
import tools, RNTK_avg
import jax
import symjax
import symjax.tensor as T
import copy

In [2]:
Ti = 2
TiPrime = 2

In [3]:
dataset = "trains"

In [4]:
dic = {k:v for k,v in map(lambda x : x.split(), open("data" + "/" + dataset + "/" + dataset + ".txt", "r").readlines())}

In [5]:
N = int(dic["n_patrons1="])
length = int(dic["n_entradas="])

In [6]:
class RNTKTEST():
    def __init__(self):
        self.sw = 1
        self.su = 1
        self.sb = 1
        self.sh = 1
        self.L = 1
        self.Lf = 0
        self.sv = 1
        self.N = int(dic["n_patrons1="])
        self.length = int(dic["n_entradas="])
        
    def VT(self, M):
        A = T.diag(M)  # GP_old is in R^{n*n} having the output gp kernel
        # of all pairs of data in the data set
        B = A * A[:, None]
        C = T.sqrt(B)  # in R^{n*n}
        D = M / C  # this is lamblda in ReLU analyrucal formula
        E = T.clip(D, -1, 1)  # clipping E between -1 and 1 for numerical stability.
        F = (1 / (2 * np.pi)) * (E * (np.pi - T.arccos(E)) + T.sqrt(1 - E ** 2)) * C
        G = (np.pi - T.arccos(E)) / (2 * np.pi)
        return F,G

In [7]:
rntk = RNTKTEST()

In [8]:
DATA = T.Placeholder((N, length), 'float32', name = "X")
x = DATA[:,0]
X = x*x[:, None]
n = X.shape[0]

In [9]:
RNTK,GP = RNTK_avg.RNTK.RNTK_first(rntk,DATA[:,0])

In [10]:
l = 1

## LAYER ZERO ATTEMPT


In [11]:
switch_flag = 1
dim_1_i = dim_1 = 2
dim_2 = 2
dim_2_i = 0
dim_num = dim_1 + dim_2 + 1

In [12]:
tiprimes = []
tis = []

print(dim_1_i, dim_2_i, switch_flag)
for d in range(0,dim_num):
    tiprime = dim_1_i
    ti = dim_2_i

    # diag_func(tiprime, ti, dim_1, dim_2, rntk)
    tiprimes.append(tiprime)
    tis.append(ti)

    if dim_1_i == 0:
        switch_flag -= 1
    else:
        dim_1_i = dim_1_i - 1
    if switch_flag <= 0:
        dim_2_i = dim_2_i + 1

2 0 1


In [13]:
np.array(list(zip(tiprimes, tis))).shape ## input size (iterable)

(5, 2)

In [14]:
np.array([dim_1, dim_2, n, X]).shape ## additional input size (unchanging)

(4,)

In [15]:
-np.ones([dim_num,min(dim_1,dim_2)], dtype = object) ## the carry

array([[-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1],
       [-1, -1]], dtype=object)

## below code DOES NOT WORK

        print("inner iteration - ", tiprime_iter, ti_iter) #// DIM 1 IS T PRIME, DIM 2 IS T
        if T.logical_and(T.greater(ti_iter, 0), T.greater(tiprime_iter, 0)):
        if ((ti_iter > 0) & (tiprime_iter > 0)):
            S, D = rntk.VT(lambdamatrix[0, ti_iter-1, tiprime_iter-1])
            lambdamatrix[0, ti_iter, tiprime_iter] = rntk.sw ** 2 * S + rntk.su ** 2 * X + rntk.sb ** 2
            phimatrix[0, ti_iter, tiprime_iter] = lambdamatrix[0, ti_iter, tiprime_iter] + rntk.sw ** 2 * phimatrix[0, ti_iter - 1, tiprime_iter - 1] * D
        else:
            test = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(n, n) + (rntk.su ** 2) * X + rntk.sb ** 2
            test = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(n, n) + (rntk.su ** 2) + rntk.sb ** 2
            test = rntk.sh ** 2 * rntk.sw ** 2 + (rntk.su ** 2) + rntk.sb ** 2
            phimatrix[0,ti,tiprime_iter] = lambdamatrix[0,ti,tiprime_iter] = T.expand_dims(test, axis = 0) # line 2, alg 1

## lets make a scan that does a diagonal

In [16]:
dim_1 = 7
dim_2 = 5

In [86]:
# Xph = T.Placeholder((10,10), "float32", name="X")
# diagonal = T.Placeholder((2,), "float32", name="signal")
diagindex = jax.numpy.arange(0,min(dim_1, dim_2))
diag = T.Variable((diagindex), "float32", "dimension internal index")
dim1ph = T.Variable(dim_1, "float32", "dimension 1 max")
dim2ph = T.Variable(dim_2, "float32", "dimension 2 max")
dim1idxph = T.Variable(0, "float32", "dimension 1 index")
dim2idxph = T.Variable(0, "float32", "dimension 2 index")
nph = T.Variable(n, "float32", "n")
# carry = T.Placeholder((), "float32", "alpha")

## prev_vals - (2,1) - previous phi and lambda values
## idx - where we are on the diagonal
## d1idx - y value of first dimension diag start
## d2idx - x value of second dimension diag start
## d1ph - max value of first dimension
## d2ph - max value of second dimension
bc = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(n, n) + (rntk.su ** 2) + rntk.sb ** 2 ## took out an X
single_boundary_condition = T.expand_dims(T.Variable((bc), "float32", "boundary_condition"), axis = 0)
boundary_condition = T.concatenate([single_boundary_condition, single_boundary_condition])

def fn(prev_vals, idx, d1idx, d2idx, d1ph, d2ph, nph):
    tiprime_iter = d1idx + idx
    ti_iter = d2idx + idx
    prev_lambda = prev_vals[0]
    prev_phi = prev_vals[1]
    ## not boundary condition
    S, D = rntk.VT(prev_lambda)
    new_lambda = rntk.sw ** 2 * S + rntk.su ** 2 + rntk.sb ** 2 ## took out an X
    new_phi = new_lambda + rntk.sw ** 2 * prev_phi * D
    lambda_expanded = T.expand_dims(new_lambda, axis = 0)
    phi_expanded = T.expand_dims(new_phi, axis = 0)
    to_return = T.concatenate([lambda_expanded, phi_expanded])
    
    return to_return, to_return

last_ema, all_ema = T.scan(
    fn, init = boundary_condition, sequences=[diag], non_sequences=[dim1idxph, dim2idxph, dim1ph, dim2ph, nph]
)

expanded_ema = T.concatenate([T.expand_dims(boundary_condition, axis = 0), o])

f = symjax.function(diag, dim1idxph, dim2idxph, dim1ph, dim2ph, nph, outputs=expanded_ema)

In [87]:
o = f(diagindex, 0, 0, dim_1, dim_2, n)
o

array([[[[3.       , 2.       , 2.       , ..., 2.       , 2.       ,
          2.       ],
         [2.       , 3.       , 2.       , ..., 2.       , 2.       ,
          2.       ],
         [2.       , 2.       , 3.       , ..., 2.       , 2.       ,
          2.       ],
         ...,
         [2.       , 2.       , 2.       , ..., 3.       , 2.       ,
          2.       ],
         [2.       , 2.       , 2.       , ..., 2.       , 3.       ,
          2.       ],
         [2.       , 2.       , 2.       , ..., 2.       , 2.       ,
          3.       ]],

        [[3.       , 2.       , 2.       , ..., 2.       , 2.       ,
          2.       ],
         [2.       , 3.       , 2.       , ..., 2.       , 2.       ,
          2.       ],
         [2.       , 2.       , 3.       , ..., 2.       , 2.       ,
          2.       ],
         ...,
         [2.       , 2.       , 2.       , ..., 3.       , 2.       ,
          2.       ],
         [2.       , 2.       , 2.       , ..., 2.

In [20]:
# lambdamatrix = np.zeros([1, dim_1 + 1, dim_2 + 1],dtype = object)
# phimatrix = np.zeros([1, dim_1 + 1, dim_2 + 1],dtype = object)

## MISC LOOKING AT OBJECT FORMATS

In [21]:
S_new,D_new = rntk.VT(gp_new[l-1])

NameError: name 'gp_new' is not defined

In [115]:
S_new

Op(name=multiply_137, fn=multiply, shape=(10, 10), dtype=float32, scope=/default)

In [116]:
D_new

Op(name=true_divide_19, fn=true_divide, shape=(10, 10), dtype=float32, scope=/default)

In [117]:
extra = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(n, n) + rntk.su**2 * S_new + rntk.sb**2

In [118]:
new = T.expand_dims(extra,axis = 0)

In [119]:
new

Op(name=expand_dims_27, fn=expand_dims, shape=(1, 10, 10), dtype=float32, scope=/default)

In [120]:
gp_new = T.concatenate([gp_new,new]) #line 4, alg 1

In [121]:
gp_new

Op(name=concatenate_13, fn=concatenate, shape=(3, 10, 10), dtype=float32, scope=/default)

## MIDDLE RNTK OBJECT TYPES

In [122]:
T.transpose(DATA[:, 1:])

Op(name=transpose_10, fn=transpose, shape=(28, 10), dtype=float32, scope=/default)

In [123]:
T.stack([RNTK,GP])

Op(name=stack_10, fn=stack, shape=(2, 2, 10, 10), dtype=float32, scope=/default)

In [124]:
r1, r2 = T.scan(lambda a,b:middletest(a,b),sequences=[ T.transpose(DATA[:, 1:]) ], init=T.stack([RNTK,GP]))

a[0] Op(name=_rewriting_take_56, fn=_rewriting_take, shape=(2, 10, 10), dtype=float32, scope=/default)
a[1] Op(name=_rewriting_take_57, fn=_rewriting_take, shape=(2, 10, 10), dtype=float32, scope=/default)
B Placeholder(name=_19, shape=(10,), dtype=float32, scope=/default)


In [125]:
def middletest(a,b):
    # print("A", a)
    print("a[0]", a[0])
    print("a[1]", a[1])
    print("B", b)
    return a,b